In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Rapids is an open-source GPU accelerated Data Sceince and Machine Learning library, developed and mainatained by Nvidia. It is designed to be compatible with many existing CPU tools, such as Pandas, scikit-learn, numpy, etc. It enables massive acceleration of many data-science and machine learning tasks, oftentimes by a factor fo 100X, or even more

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GroupKFold, KFold
from sklearn.metrics import mean_squared_error
from tqdm.notebook import tqdm

In [ ]:
train = pd.read_csv("/kaggle/input/tabular-playground-series-jan-2021/train.csv")
test = pd.read_csv("/kaggle/input/tabular-playground-series-jan-2021/test.csv")
sample_submission = pd.read_csv('../input/tabular-playground-series-jan-2021/sample_submission.csv')

In [ ]:
target = train['target'].values
columns = test.columns[1:]

In [ ]:
rr_train_oof = np.zeros(300000,)
rr_test_preds = 0
rr_train_oof.shape

In [ ]:
NUM_FOLDS = 5

kf = KFold(n_splits = NUM_FOLDS, shuffle = True, random_state = 2021)

for f, (train_idx, val_idx) in tqdm(enumerate(kf.split(train, target))) :
    
    train_x, val_x = train.iloc[train_idx][columns], train.iloc[val_idx][columns]
    train_y, val_y = target[train_idx], target[val_idx]
    
    model = Ridge(alpha = 0.01)
    model.fit(train_x, train_y)
    temp_oof = model.predict(val_x)
    temp_test = model.predict(test[columns])
    
    rr_train_oof[val_idx] = temp_oof
    rr_test_preds += temp_test / NUM_FOLDS


In [ ]:
mean_squared_error(rr_train_oof, target, squared = False)

In [ ]:
rf_train_oof = np.zeros(3000000,)
rf_test_preds = 0
rf_train_oof.shape

In [ ]:
NUM_FOLDS = 5

kf = KFold(n_splits = NUM_FOLDS, shuffle = True, random_state = 2021)

for f, (train_idx, val_idx) in tqdm(enumerate(kf.split(train, target))) :
    train_x, val_x = train.iloc[train_idx][columns], train.iloc[val_idx][columns]
    train_y, val_y = target[train_idx], target[val_idx]
    
    model = RandomForestRegressor(n_estimators = 500, max_depth = 15)
    model.fit(train_x, train_y)
    temp_oof = model.predict(val_x)
    
    temp_test = model.predict(test[columns])
    rf_train_oof[val_idx] = temp_oof
    test_preds += temp_test /NUM_FOLDS

In [ ]:
mean_squared_error(rf_train_oof, target, squared = False)

In [ ]:
mean_squared_error(0.9*rf_train_oof + 0.1*rr_train_oof, target, squared = False)

In [ ]:
knn_train_oof = np.zeros(3000000,)
knn_test_preds = 0
knn_train_oof.shape

In [ ]:
NUM_FOLDS = 5

kf = KFold(n_splits = NUM_FOLDS, shuffle = True, random_state = 2021)

for f, (train_idx, val_idx) in tqdm(enumerate(kf.split(train, target))) :
    train_x, val_x = train.iloc[train_idx][columns], train.iloc[val_idx][columns]
    train_y, val_y = target[train_idx], target[val_idx]
    
    model = KNeighborsRegressor(n_neighbors = 150)
    model.fit(train_x, train_y)
    temp_oof = model.predict(val_x)
    
    temp_test = model.predict(test[columns])
    knn_train_oof[val_idx] = temp_oof
    knn_test_preds += temp_test /NUM_FOLDS

In [ ]:
mean_squared_error(knn_train_oof, target, squared = False)

In [ ]:
mean_squared_error(0.5*rf_train_oof+0.5*knn_train_oof, target, squared=False)

In [ ]:
svr_train_oof = np.zeros(3000000,)
svr_test_preds = 0
svr_train_oof.shape

In [ ]:
NUM_FOLDS = 5

kf = KFold(n_splits = NUM_FOLDS, shuffle = True, random_state = 2021)

for f, (train_idx, val_idx) in tqdm(enumerate(kf.split(train, target))) :
    train_x, val_x = train.iloc[train_idx][columns], train.iloc[val_idx][columns]
    train_y, val_y = target[train_idx], target[val_idx]
    
    model = SVR(C = 0.1)
    model.fit(train_x, train_y)
    temp_oof = model.predict(val_x)
    
    temp_test = model.predict(test[columns])
    svr_train_oof[val_idx] = temp_oof
    svr_test_preds += temp_test /NUM_FOLDS

In [ ]:
mean_squared_error(svr_train_oof, target, squared = False)

In [ ]:
mean_squared_error(0.35*rf_train_oof + 0.34*knn_train_oof+0.31*svr_train_oof, target, squared = False)

In [ ]:
sample_submission['target'] = 0.35*rf_test_preds+0.34*knn_test_preds+0.31*svr_test_preds
sample_submission.to_csv('submission.csv', index=False)